# Tiền xử lí

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!mkdir data

!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2018_post.csv" -O data/ground_water_quality_2018_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2019_post.csv" -O data/ground_water_quality_2019_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2020_post.csv" -O data/ground_water_quality_2020_post.csv

!ls data/

In [14]:
import os 
BASE_DIR = os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath('notebook/preprocessing.ipynb'))))

path18 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2018_post.csv')
path19 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2019_post.csv')
path20 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2020_post.csv')

In [12]:
df_18 = pd.read_csv(path18)
df_19 = pd.read_csv(path19)
df_20 = pd.read_csv(path20)

## Chuẩn hóa tên cột

In [16]:
df_19.rename(columns={'EC': 'E.C', 'CO_-2 ': 'CO3', 'HCO_ - ': 'HCO3', 'Cl -': 'Cl', 'F -': 'F', 'NO3- ': 'NO3', 'SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'}, inplace=True)
df_18.rename(columns={'NO3 ': 'NO3'}, inplace=True)
df_20.rename(columns={'NO3 ': 'NO3'}, inplace=True)

## Chuẩn hóa giá trị

### Thống nhất class O.G năm 2020 và OG năm 2019 là OG

In [17]:
df_20 = pd.concat(
    [df_20.iloc[:, :23], 
     df_20['Classification'].replace('O.G', 'OG'), 
     df_20.iloc[:, 24:]
    ], 
    axis=1)

### Xử lí null

In [22]:
len(df_18), len(df_19), len(df_20)

(374, 364, 368)